In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_datareader.data import DataReader
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import datetime
import os
import glob as glob

In [2]:
# This function reads the most recent dataset and cuts this based on the date range provided 

def cut_raw_data(start_date, end_date):
    
    # Set base path
    base_path = r'C:\Users\mball3\OneDrive - KPMG\Documents\Investment_Analysis\data'

    # This returns all folders in the base_path
    # return the folder, in the base path, if it is a folder
    # And only return folders of length 10 containins two underscores 
    folders = [folder for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))
               and len(folder) == 10 and folder.count('_') == 2
              ]

    # Find latest folder in the path
    latest_folder = max(folders, key=lambda x: datetime.strptime(x,'%d_%m_%Y'))

    # Create new full path to the most recent folder
    latest_path = os.path.join(base_path, latest_folder)

    # Retrieve the file names
    filenames = glob.glob(latest_path+"/*.csv")

    # Combine data for all stocks
    all_data = []
    for filename in filenames:

        # Read in the historical data for each stock 
        historical_data = pd.read_csv(filename)
        Stock = filename.split("\\")[-1].split('.')[0]

        # Convert date to date format - english 
        historical_data['Stock'] = Stock
        historical_data['Date'] = pd.to_datetime(historical_data['Date'])

        historical_data.sort_values(by=['Date'], inplace=True)

        # Store the data
        all_data.append(historical_data)

    # Concatenate the data    
    df = pd.concat(all_data, ignore_index=True)

    # Convert date to date format 
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(by=['Date'], inplace=True)
    
    # Cut by dates
    df = df[(df['Date'] > start_date) & (df['Date'] < end_date) ]

    # Set the date as the index
    df = df.set_index('Date')

    # make certain columns numeric 
    cols_to_convert = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

    # Confirm column names
    df.columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Stock']
    
    return df


In [3]:
# This is the date range I had initally analysed
# It was just over a year's worth of data from the first data i began my analysis
df = cut_raw_data(start_date='2022-12-18', end_date='2024-01-15')
df

,Open,High,Low,Close,Adj Close,Volume,Stock
Date,,,,,,,
2022-12-19,135.110001,135.199997,131.320007,132.369995,131.634232,79592600,AAPL
2022-12-19,442.329987,442.850006,432.100006,438.690002,433.406830,1064500,LRCX
2022-12-19,86.000000,86.620003,85.430000,86.419998,83.206001,5834800,GILD
2022-12-19,116.830002,117.800003,114.330002,114.480003,114.480003,29769900,META
2022-12-19,121.559998,121.690002,120.360001,120.790001,120.046349,1606700,EA
...,...,...,...,...,...,...,...
2024-01-12,47.970000,47.970000,47.380000,47.570000,47.570000,3672300,CPRT
2024-01-12,155.390000,156.200000,154.010000,154.620000,154.620000,40460300,AMZN
2024-01-12,186.060000,186.740000,185.190000,185.920000,185.920000,40444700,AAPL
